In [360]:
import dask
from fake_useragent import UserAgent
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import pandas as pd
import json
import time
import bookquery
import os
import io
import json
from tqdm import tqdm
from os.path import join,isfile
from os import listdir
from collections import Counter
import dask

ua = UserAgent()

In [362]:

def execute_query(qquery, entity_id):
    ua = UserAgent()
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def entities_sparqul(entity_id, f = 0):
    res = []
    results= execute_query(bookquery.entity_finding, entity_id)
    for result in results:
        entities = {}
        entities['itemid'] = result['item']['value'].split('/')[-1]
        entities['label'] = result['itemLabel']['value']
        res.append(entities)
    return pd.DataFrame(data = res, columns = ['itemid', 'label'])
        

In [28]:
df_items = entities_sparqul("Q119253")
df_items

,itemid,label
0,Q119196,Кермадек
1,Q1423293,Yap Trench
2,Q3079494,Beaufort's Dyke
3,Q3373484,Hikurangi Trench
4,Q5434800,Farallon Trench
5,Q118873,Перуанско-Чилийский жёлоб
6,Q164040,Центральноамериканский жёлоб
7,Q9325582,Timor Trough
8,Q56319989,Q56319989
9,Q118537,Японский жёлоб


In [29]:
def execute_query_2(qquery, entity_id , pred_id):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id , pred_id =pred_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

# def predicates(entity_id,pred_id):
#     res = []
#     results= execute_query(bookquery.entity_finding, entity_id)
#     for result in results:
#         entities = {}
#         entities['item'] = result['item']['value'].split('/')[-1]
#         entities['label'] = result['itemLabel']['value']
#         entities['answer'] = result['answer']['value']
#         res.append(entities)
#     return pd.DataFrame(data = res, columns = ['item', 'label' , 'answer'])


def predicates(entity_id, pred_id, label, f = 0):
        res = []
        if f > 5:
            return None
        try:
            results= execute_query_2(bookquery.predicat_reciver, entity_id , pred_id)
            if len(results)>1:
                results = [results[0]]
            entities = {}
            for result in results:
                entities['itemid'] = result['item']['value'].split('/')[-1]
                entities['label'] = result['itemLabel']['value']
                entities['answer'] = result['answer']['value']
                entities['point'] = 1
                res.append(entities)
            if len(res) == 0:
                entities['itemid'] = entity_id
                entities['label'] = label
                entities['answer'] = None
                entities['point'] = 0
                res.append(entities)
            return pd.DataFrame(data = res, columns = ['itemid', 'label' , 'answer' , "point"])
        except Exception as e:
            print(e)
            time.sleep(5)
            print('Try again ', entity_id, pred_id, f)
            return predicates(entity_id,pred_id, label ,f + 1)

In [366]:
def counter(entity_id):
    results= execute_query(bookquery.subclass_entity_count, entity_id)
    for result in results:
        answer = result['entitycount']['value']
    return answer

def answers(entity_id, pred_id):
    res = []
    df_items = entities_sparqul(entity_id)
    for k,v in df_items.iterrows():
        res.append(dask.delayed(predicates)(v.itemid , pred_id , v.label))
    result = dask.compute(*res)
    final_result = pd.concat(result)
    final_result
    return final_result,counter(entity_id)

In [598]:
df,count = answers("Q15310171", "P17")
summa = round(df.point.sum()/len(df),2) * 100
if len(df)<50:
    print(f'Процент ответов по всем значениям:{summa} %')
else:
    print(f'Процент ответов по выборке из {count} представителей:{summa} %')
df

Процент ответов по выборке из 127 представителей:92.0 %


,itemid,label,answer,point
0,Q3964,Гамбургский порт,http://www.wikidata.org/entity/Q183,1
0,Q16460,Port of Santos,http://www.wikidata.org/entity/Q155,1
0,Q437959,Old Port of Marseille,http://www.wikidata.org/entity/Q142,1
0,Q509588,Порт Джэбэль Али,http://www.wikidata.org/entity/Q878,1
0,Q858692,"Big Creek, Belize",http://www.wikidata.org/entity/Q242,1
0,Q883613,Port of Emden,http://www.wikidata.org/entity/Q183,1
0,Q935686,Ларн,http://www.wikidata.org/entity/Q145,1
0,Q1349796,Port of Barcelona,http://www.wikidata.org/entity/Q29,1
0,Q1552633,Q1552633,None,0
0,Q1567868,Порт Гданьск,http://www.wikidata.org/entity/Q36,1
